In this notebook will be demonstrating how to train using convolutional networks using TensorStream and its OpenCL backend.

Note that code here is based on Martin Gorner's talk in "TensorFlow and Deep Learning without a PhD, Part 1 (Google Cloud Next '17)""

https://www.youtube.com/watch?v=u4alGiomYP4

Include the OpenCL backend as working with images is compute intensive. Note that this requires OpenCL to be configured properly on your machine.

In [1]:
require 'tensor_stream'
require 'mnist-learn'
require 'csv'

require 'tensor_stream/opencl'

ts = TensorStream
puts "Tensorstream version #{ts.__version__} with OpenCL lib #{TensorStream::Opencl::VERSION}"

Warning OpenCL 2.0 loader detected!


Tensorstream version 1.0.0 with OpenCL lib 0.2.8


Download the MNIST data set which we will use for training the network

In [2]:
# Import MNIST data
puts "downloading minst data"
# Download images and labels into mnist.test (10K images+labels) and mnist.train (60K images+labels)
mnist = Mnist.read_data_sets('/tmp/data', one_hot: true)

puts "downloading finished"

downloading minst data
downloading finished


Setup parameters that we will use for the network:

In [3]:
K = 4 # first convolutional layer output depth
L = 8 # second convolutional layer output depth
M = 12 # third convolutional layer
N = 200 # fully connected layer
EPOCH = 2000

2000

Setup placeholders. Placeholders are like input parameters that your model can accept and has no definite value until you give it one during sess.run

In [4]:
# input X: 28x28 grayscale images, the first dimension (None) will index the images in the mini-batch
x = ts.placeholder(:float32, shape: [nil, 28, 28, 1])

# correct answers will go here
y_ = ts.placeholder(:float32, shape: [nil, 10])

# step for variable learning rate
step_ = ts.placeholder(:int32)

pkeep = ts.placeholder(:float32)

Placeholder(Placeholder_4 shape: ? data_type: float32)

Here we declare variables. The contents of these variables are randomized initially, however this get updated automatically during training. Variables contain the weights of the network and the values serves as neural connections that make the system learn.

In [5]:
w1 = ts.variable(ts.truncated_normal([6, 6, 1, K], stddev: 0.1))
b1 = ts.variable(ts.ones([K])/10)

w2 = ts.variable(ts.truncated_normal([5, 5, K, L], stddev: 0.1))
b2 = ts.variable(ts.ones([L])/10)

w3 = ts.variable(ts.truncated_normal([4, 4, L, M], stddev: 0.1))
b3 = ts.variable(ts.ones([M])/10)

w4 = ts.variable(ts.truncated_normal([7 * 7 * M, N], stddev: 0.1))
b4 = ts.variable(ts.ones([N])/10)

w5 = ts.variable(ts.truncated_normal([N, 10], stddev: 0.1))
b5 = ts.variable(ts.ones([10])/10)


Variable(Variable_10:0 shape: TensorShape([Dimension(10)]) data_type: float32)

Here we declare the model itself. These define the computation that make up the structure of the neural network. In this case we are setting up 3 convolutional layers and 2 fully connected layers. We are also using relu as the activiation functions. The kinds of functions to use are based on decades of research and this can change depending on new findings.

In [6]:
# The model
stride = 1  # output is 28x28
y1 = ts.nn.relu(ts.nn.conv2d(x.reshape([-1, 28, 28, 1]), w1, [1, stride, stride, 1], 'SAME') + b1)
stride = 2  # output is 14x14
y2 = ts.nn.relu(ts.nn.conv2d(y1, w2, [1, stride, stride, 1], 'SAME') + b2)
stride = 2  # output is 7x7
y3 = ts.nn.relu(ts.nn.conv2d(y2, w3, [1, stride, stride, 1], 'SAME') + b3)

# reshape the output from the third convolution for the fully connected layer
yy = y3.reshape([-1, 7 * 7 * M])
y4 = ts.nn.relu(ts.matmul(yy, w4) + b4)

ylogits = ts.matmul(y4, w5) + b5

# model
y = ts.nn.softmax(ylogits, name: 'out')

Op(softmax name: out shape: ? data_type: float32)

Now we define the error function to use and the optimization algorithm. There are various error functions to choose from as well as optimization algorithms, most have their pros and cons. However for this type of neural network the softmax cross entropy and the Adam optimizer seems the most appropriate.

In [7]:
cross_entropy = ts.nn.softmax_cross_entropy_with_logits(logits: ylogits, labels: y_)
cross_entropy = ts.reduce_mean(cross_entropy)*100

is_correct = ts.equal(ts.argmax(y, 1), ts.argmax(y_, 1))
accuracy =  ts.reduce_mean(is_correct.cast(:float32))

# training step, learning rate = 0.003
lr = 0.0001.t +  ts.train.exponential_decay(0.003, step_, 2000, 1/Math::E)
train_step = TensorStream::Train::AdamOptimizer.new(lr).minimize(cross_entropy)

Op(flow_group name: Adam/flow_group shape: TensorShape([Dimension(12)]) data_type: )

Setup test data and use a saver so that progress can be continued on the next run. Here we also
initialize the variables, otherwise they will contain null values and cause errors during the next sess.run

In [10]:
sess = ts.session
# Add ops to save and restore all the variables.

init = ts.global_variables_initializer

sess.run(init)

#Setup save and restore
model_save_path = "test_models/mnist_data_3.0"
saver = TensorStream::Train::Saver.new
saver.restore(sess, model_save_path)

mnist_train = mnist.train
test_data = { x => mnist.test.images, y_ => mnist.test.labels, pkeep => 1.0 }

nil

Interrupt: 

In [9]:
(0..EPOCH).each do |i|
  # load batch of images and correct answers
  batch_x, batch_y = mnist_train.next_batch(100)
  train_data = { x => batch_x, y_ => batch_y, step_ => i, pkeep => 0.75 }

  # train
  sess.run(train_step, feed_dict: train_data)

  if (i % 10 == 0)
    # result = TensorStream::ReportTool.profile_for(sess)
    # File.write("profile.csv", result.map(&:to_csv).join("\n"))
    # success? add code to print it
    a_train, c_train, l = sess.run([accuracy, cross_entropy, lr], feed_dict: { x => batch_x, y_ => batch_y, step_ => i, pkeep => 1.0})
    puts "#{i}: accuracy:#{a_train} loss:#{c_train} (lr:#{l})"
  end

  if (i % 100 == 0)
    # success on test data?
    a_test, c_test = sess.run([accuracy, cross_entropy], feed_dict: test_data, pkeep => 1.0)
    puts("#{i}: ******** test accuracy: #{a_test} test loss: #{c_test}")

    # save current state of the model
    save_path = saver.save(sess, model_save_path)
  end
end



0: accuracy:0.9599999785423279 loss:15.933753967285156 (lr:0.003100000089034438)
0: ******** test accuracy: 0.9283999800682068 test loss: 2699.62841796875
10: accuracy:0.949999988079071 loss:11.798951148986816 (lr:0.003085037227720022)
20: accuracy:0.9399999976158142 loss:13.785243034362793 (lr:0.003070149337872863)
30: accuracy:0.9300000071525574 loss:29.51731300354004 (lr:0.003055335721001029)
40: accuracy:0.9700000286102295 loss:7.979790687561035 (lr:0.0030405959114432335)
50: accuracy:0.9300000071525574 loss:22.561899185180664 (lr:0.0030259296763688326)
60: accuracy:0.9700000286102295 loss:17.135129928588867 (lr:0.003011336550116539)
70: accuracy:0.949999988079071 loss:12.989716529846191 (lr:0.002996816299855709)
80: accuracy:0.9599999785423279 loss:23.976905822753906 (lr:0.002982368227094412)
90: accuracy:0.8999999761581421 loss:25.88566017150879 (lr:0.002967992564663291)
100: accuracy:0.9700000286102295 loss:9.034774780273438 (lr:0.002953688381239772)
100: ******** test accuracy:

930: accuracy:1.0 loss:0.5913202166557312 (lr:0.0019844050984829664)
940: accuracy:1.0 loss:1.3453789949417114 (lr:0.0019750066567212343)
950: accuracy:0.9900000095367432 loss:1.7747830152511597 (lr:0.0019656552467495203)
960: accuracy:1.0 loss:1.236012578010559 (lr:0.0019563501700758934)
970: accuracy:1.0 loss:1.853969693183899 (lr:0.0019470915431156754)
980: accuracy:1.0 loss:0.845487117767334 (lr:0.0019378792494535446)
990: accuracy:0.9900000095367432 loss:1.9294251203536987 (lr:0.00192871259059757)
1000: accuracy:1.0 loss:0.9650002121925354 (lr:0.001919591915793717)
1000: ******** test accuracy: 0.9814000129699707 test loss: 1020.56982421875
1010: accuracy:0.9599999785423279 loss:16.393333435058594 (lr:0.0019105166429653764)
1020: accuracy:0.9900000095367432 loss:3.5150673389434814 (lr:0.0019014865392819047)
1030: accuracy:1.0 loss:0.7717859745025635 (lr:0.0018925016047433019)
1040: accuracy:0.9700000286102295 loss:6.396111965179443 (lr:0.0018835614901036024)
1050: accuracy:0.99000

1910: accuracy:1.0 loss:0.2219347357749939 (lr:0.0012544363271445036)
1920: accuracy:0.9900000095367432 loss:5.134439468383789 (lr:0.0012486785417422652)
1930: accuracy:1.0 loss:0.9207942485809326 (lr:0.001242949510924518)
1940: accuracy:1.0 loss:0.47080865502357483 (lr:0.0012372490018606186)
1950: accuracy:1.0 loss:1.2955037355422974 (lr:0.0012315770145505667)
1960: accuracy:0.9900000095367432 loss:2.417428493499756 (lr:0.0012259331997483969)
1970: accuracy:1.0 loss:0.6553899049758911 (lr:0.0012203175574541092)
1980: accuracy:1.0 loss:0.49434810876846313 (lr:0.00121472985483706)
1990: accuracy:0.9900000095367432 loss:1.8686177730560303 (lr:0.0012091703247278929)
2000: accuracy:1.0 loss:0.46403393149375916 (lr:0.001203638268634677)
2000: ******** test accuracy: 0.9843000173568726 test loss: 821.3646240234375


0..2000